In [28]:
import sympy as sm
import numpy as np
from scipy import optimize
from scipy import interpolate
from bokeh.io import output_notebook,push_notebook,show
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter

import ipywidgets as widgets
from ipywidgets import interact 

output_notebook()


Loading BokehJS ...

In [30]:
def total_utility(c, weight, theta):
    '''
    Sums utility for c for multiple years
    c is an array 
    '''
    if theta == 1:
        uts =np.log(c)
    else:
        uts = (c**(1-theta)-1)/(1-theta)
    # sum of utitity
    t_u = np.dot(uts,weight)

    return t_u

def prod(k,l,alpha,a):
    return (k**alpha)*((a*l)**(1-alpha))

def tot_ut_multiple_sks_quick(sks, k0, l, b, weight, theta, alpha, delta):
    '''
    Finds total utitilty for a set of years with a savingsrate for each year
    '''
    t = len(sks)
    k_short = np.empty(t)
    k_short[0] = k0
    
    for i in range(1,t):    
        k_short[i]=sks[i-1]*prod(k_short[i-1],l[i-1],alpha,b)+(1-delta)*k_short[i-1]
    
    y_short = prod(k_short,l,alpha,b)
    
    return total_utility(y_short*(1-sks)/l, weight, theta)


In [32]:
def optimal_sks(t, b, l, weight, delta, alpha, theta, k0, first=True):
    '''
    Optimizes the utility of the representative household
    by simulating the predicted future
    '''
    obj = lambda sks: -tot_ut_multiple_sks_quick(sks, k0, l, b, weight, theta, alpha, delta)
    sks0 = np.linspace(alpha,0,t)

    bounds = np.full((t,2),[1e-8,0.99999])
    res = optimize.minimize(obj, sks0, method='SLSQP', 
        bounds=bounds,)
    
    if res.success:
        if first:
            return res.x[0]
        else:
            return res.x  
    else:
        print('Optimization was sadly not succesfull')

In [34]:
def find_ssk_sk(k,b,delta,n,alpha):
    return (k**(1-alpha)*(delta+n))/b

In [36]:
def solving_and_plotting_interactive_micro(
        t, b, n, weight, delta, alpha, l, k0s_bounds=[0.001,30], theta_bounds=[0.3,1.5], precision=100,
        width=1000, height=600): 
    
    # data:
    k0s = np.linspace(k0s_bounds[0],k0s_bounds[1],precision)
    sks_plot = np.zeros(precision)
    theta = 0.5
    for i in range(k_n):
            sks_plot[i] = optimal_sks(t, b, l, weight, delta, alpha, theta, k0s[i])
    
    
    steadystates = find_ssk_sk(k0s,b,delta,n,alpha)
    
    '''
    Bokeh plotting
    '''
     # Bokeh needs a name for the data that neither has spaces nor numbers
    # because we want the option to do this we define abitrairy calls via the alphabeth. 
    
    

    tooltips=[('Capital pr. capita','@x{0,0.00}'),('Savings rate in steady state','@y{0.00}'),
             (f'Savingsrate for \u03B8 = {theta}','@z{0.00}')]
    
    
    hover = HoverTool(tooltips=tooltips)
    tools="pan,wheel_zoom,box_zoom,reset,save"
    
    
    data = {'x' : k0s,'y' : steadystates, 'z' : sks_plot}
        
    source = ColumnDataSource(data)
    
    p = figure(plot_width=width, plot_height=height, title=f'Optimal savings rate for varying \u03B8 and capital pr. capita', 
        tools=[hover,tools], x_axis_label=f'Capital pr. capita', y_axis_label=f'Savings rate')

    
    p.line(x='x', y='y', source=source, legend= 'Steady state', color = 'yellow')
    p.line(x='x', y='z', source=source, legend= f'\u03B8 = {theta}', color = 'blue')
    
    p.legend.location = 'top_center'
    
    
    def update_theta(new_theta):
        theta = new_theta
        
        # push_notebook is af bokeh->jupyter-specific comand to tell python to update the bokeh plot in jupyter
        push_notebook()
    
    # show the plot, notebook_handle is again bokeh->jupyter-specific to show the plot in the notebook
    show(p,notebook_handle=True)

    # Make a dropdown-widget, so the user can choose different stocks, the layout option, makes the options box wider
    # the style options make the desciption box wide enough for the description string to be read fully
    drop_down = widgets.Dropdown(options=stock_list, layout = {'width':'50%'},\
        description='Choose a stock or index:',style = {'description_width': 'initial'})

    # Call the function 
    interact(update_name, stock = drop_down)
    
    
    return p 


**Interactive Micro**

In [65]:
# Parameters
# Macro:

alpha = 1/3
delta = 0.05
n = 0.008
b = 1
k0 = 5
l0 = 1

#Micro:
theta = 0.5
beta = 0.99

t = 100

# time preference utility weights and population growth 
# are precomputed to save time:
weight = np.array([beta**i for i in range(t)])
l = np.array([l0*(1+n)**i for i in range(t)])

In [67]:
k0s_bounds=[0.001,30] 
theta_bounds=[0.3,1.5]
precision=50
width=600
height=600
tools="pan,wheel_zoom,box_zoom,reset,save"

In [69]:
# data:
k0s = np.linspace(k0s_bounds[0],k0s_bounds[1],precision)
sks_plot = np.zeros(precision)
theta = 0.5
for i in range(precision):
        sks_plot[i] = optimal_sks(t, b, l, weight, delta, alpha, theta, k0s[i])
    
    
steadystates = find_ssk_sk(k0s,b,delta,n,alpha)
    


tooltips=[('Capital pr. capita','@x{0,0.00}'),('Savings rate in steady state','@y{0.00}'),
            (f'Optimal savingsrate','@z{0.00}')]
    
    
hover = HoverTool(tooltips=tooltips)
    
data = {'x' : k0s,'y' : steadystates, 'z' : sks_plot,'theta_legend': [f'\u03B8 = {theta}' for i in range(precision)]}
        
source = ColumnDataSource(data)
    
p = figure(plot_width=width, plot_height=height, title=f'Optimal savings rate for varying \u03B8 and capital pr. capita', 
        tools=[hover,tools], x_axis_label=f'Capital pr. capita', y_axis_label=f'Savings rate')

    
p.line(x='x', y='y', source=source, legend= 'Steady state', color = 'yellow')
p.line(x='x', y='z', source=source, legend= 'theta_legend', color = 'blue')
    
p.legend.location = 'top_center'
    
    
def update_theta(Theta=0.5):
    
    for i in range(precision):
        sks_plot[i] = optimal_sks(t, b, l, weight, delta, alpha, Theta, k0s[i])
    data['z']=sks_plot
    data['theta_legend']= [f'\u03B8 = {Theta}' for i in range(precision)]
    source.data = ColumnDataSource(data).data
    
    push_notebook()
    
    
show(p,notebook_handle=True)

interact(update_theta, Theta = (0,6,0.1))

interactive(children=(FloatSlider(value=0.5, description='Theta', max=6.0), Output()), _dom_classes=('widget-i…

<function __main__.update_theta(Theta=0.5)>

Too slow